In [1]:
import functools
import abc

In [2]:
class Spam(metaclass=abc.ABCMeta):

    @abc.abstractmethod
    def some_method(self):
        raise NotImplemented()


class Eggs(Spam):
    def some_new_method(self):
        pass

In [3]:
eggs = Eggs()

TypeError: Can't instantiate abstract class Eggs with abstract method some_method

In [4]:
class Bacon(Spam):
    def some_method():
        pass

In [5]:
bacon = Bacon()

In [6]:
class Spam(object, metaclass=abc.ABCMeta):
    @property
    @abc.abstractmethod
    def some_property(self):
        raise NotImplemented()

    @classmethod
    @abc.abstractmethod
    def some_classmethod(cls):
        raise NotImplemented()

    @staticmethod
    @abc.abstractmethod
    def some_staticmethod():
        raise NotImplemented()

    @abc.abstractmethod
    def some_method():
        raise NotImplemented()

In [7]:
class AbstractMeta(type):
    def __new__(metaclass, name, bases, namespace):
        cls = super().__new__(metaclass, name, bases, namespace)
        cls.__abstractmethods__ = frozenset(('something',))
        return cls


class Spam(metaclass=AbstractMeta):
    pass

In [9]:
eggs = Spam()

TypeError: Can't instantiate abstract class Spam with abstract method something

In [14]:
class AbstractMeta(type):
    def __new__(metaclass, name, bases, namespace):
        # Create the class instance
        cls = super().__new__(metaclass, name, bases, namespace)

        # Collect all local methods marked as abstract
        abstracts = set()
        for k, v in namespace.items():
            if getattr(v, '__abstract__', False):
                abstracts.add(k)

        # Look for abstract methods in the base classes and add them to the list of abstracts
        for base in bases:
            for k in getattr(base, '__abstracts__', ()):
                v = getattr(cls, k, None)
                if getattr(v, '__abstract__', False):
                    abstracts.add(k)

        # store the abstracts in a frozenset so they cannot be modified
        cls.__abstracts__ = frozenset(abstracts)

        # Decorate the __new__ function to check if all abstract functions were implemented
        original_new = cls.__new__

        @functools.wraps(original_new)
        def new(self, *args, **kwargs):
            for k in self.__abstracts__:
                v = getattr(self, k)
                if getattr(v, '__abstract__', False):
                    raise RuntimeError(
                        '%r is not implemented' % k)

            return original_new(self, *args, **kwargs)

        cls.__new__ = new
        return cls


def abstractmethod(function):
    function.__abstract__ = True
    return function


class Spam(metaclass=AbstractMeta):
    @abstractmethod
    def some_method(self):
        # pass
        # return "shit"
        raise NotImplemented()

In [15]:
eggs = Spam()

RuntimeError: 'some_method' is not implemented